**Stile read me file**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install nltk
!pip install tensorflow_text
!pip install tensorflow_hub
!pip install pandas
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.16.1 requires tensorflow<2.17,>=2.16.1; platform_machine != "arm64" or pl

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.9 MB/s eta 0:00:00


**Versione definitiva**



In [ ]:
import os
import re
import pandas as pd
import torch
import nltk
from nltk.tokenize import sent_tokenize
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm #library that creates a progression bar in the terminal

# Download BERT LaBSE model
tokenizer_labse = BertTokenizer.from_pretrained("sentence-transformers/LaBSE")
model = BertModel.from_pretrained("sentence-transformers/LaBSE", device_map = 'cuda') #cuda allows me to use GPU with Pytorch
nltk.download('punkt') # nltk's sentencizer (module) that splits sentences when it encounters a punctuation sign

# Function to split text into sentences using the nltk's sentencizer
def split_into_sentences(text):
    return sent_tokenize(text)

# Function to get the LABSE embeddings for each sentence
def get_labse_embedding(sentences):
    tokens = tokenizer_labse(sentences, return_tensors='pt', padding=True, truncation=True) # To obtain our sentence embeddings, we need to turn our sentences into vector representations, our tensors. In order to be processed by the LaBSE Bert-basedmodel, padding is required to uniform sentence length and process them in parallel batches
    tokens = {k: tokens[k].to('cuda') for k in tokens.keys()} #We have a set of tensors, each with a label (key). To perform computations swiftly, we have to use a powerful machine (GPU). The code takes each tensor, sends it to the powerful machine, and updates the set to reflect this move. The code iterates over all items, sends each one to the fast machine, and updates the collection accordingly.
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze() # This ensures that we get a single embedding of a fixed size for each sentence.

# Function to calculate cosine similarity
def calculate_cosine_similarity(embeddings1, embeddings2):
    similarity = cosine_similarity(embeddings1, embeddings2)
    return similarity

# Function to process the subfolder corresponding to a single entry in multiple languages.
# It also initializes data structures (a dictionary) to store sentences by language, along with a variable to track the maximum number of sentences.
def process_subdirectory(directory):
    files = os.listdir(directory)
    sentences_by_language = {}
    max_sentences = 0

    # Raggruppa i testi per lingua e memorizza il numero massimo di frasi
    for file in files:
        match = re.search(r'_([a-zA-Z]{2})\.txt$', file)
        if match:
            language = match.group(1)
            if language not in sentences_by_language:
                sentences_by_language[language] = []
            with open(os.path.join(directory, file), 'r', encoding='utf-8') as f:
                text = f.read()
                sentences = split_into_sentences(text)
                sentences_by_language[language].extend(sentences)
                # Aggiorna il numero massimo di frasi
                max_sentences = max(max_sentences, len(sentences))

    # Calcola gli embeddings per ogni lingua
    embeddings_by_language = {}
    for language, sentences in sentences_by_language.items():
        sentence_embeddings = get_labse_embedding(sentences) # troviamo i tensori degli embedding delle frasi
        zero_tensor = torch.zeros(sentence_embeddings[0].shape[0]) #To ensure uniform length across languages, we create a 0-tensor with the same dimension as the other embeddings for padding purposes. In this way all languages have the same number of sentence embeddings
        pad_length = (max_sentences - len(sentences)) # troviamo la lunghezza di padding
        pad_list = [zero_tensor] * pad_length # costruiamo la lista non-tensor di tensori
        if pad_list:
            pad_tensor = torch.vstack(pad_list).to('cuda') # se la lista ha elementi creiamo il tensore (vstack non funziona con liste vuote)
            embeddings_by_language[language] = torch.vstack([sentence_embeddings, pad_tensor]) # facciamo lo stack verticale (vstack appunto) degli embeddings e dei 0-tensors
        else:
            embeddings_by_language[language] = sentence_embeddings # se la lista è vuota mettiamo solo gli embedding nel dizionario 'embeddings_by_language'


    # Calcola la cosine similarity tra tutte le coppie di frasi
    similarities = []
    languages = list(embeddings_by_language.keys())
    for i in range(len(languages)):
        lang1 = languages[i]
        for j in range(i + 1, len(languages)):
            lang2 = languages[j]
            similarity = calculate_cosine_similarity(embeddings_by_language[lang1].cpu(), embeddings_by_language[lang2].cpu())
            for k in range(similarity.shape[0]):
                for l in range(similarity.shape[1]):
                    if k < len(sentences_by_language[lang1]) and l < len(sentences_by_language[lang2]):
                        similarities.append({
                            'Language 1': lang1,
                            'Sentence 1': sentences_by_language[lang1][k],
                            'Language 2': lang2,
                            'Sentence 2': sentences_by_language[lang2][l],
                            'Cosine similarity': similarity[k][l]
                        })

    # Crea un DataFrame con i risultati e restituiscilo
    df = pd.DataFrame(similarities)
    return df

# Funzione principale
def main():
    main_directory = "/content/drive/MyDrive/Wiki download/Gastr_L_puliti/Italian_cuisine"               ##"./data"
    subdirectories = os.listdir(main_directory)

    sim_dir = "/content/drive/MyDrive/Wiki download/Gastr_L_puliti"              ##'./similarities'

    if not os.path.exists(sim_dir):
        os.makedirs(sim_dir)

    progbar = tqdm(subdirectories, total = len(subdirectories))
    for subdirectory in progbar:
        progbar.set_description(desc=f'Current subdirectory: {subdirectory}') # scriviamo la lingua corrente nella progress bar
        subdirectory_path = os.path.join(main_directory, subdirectory)
        if os.path.isdir(subdirectory_path):
            df = process_subdirectory(subdirectory_path)
            csv_filename = f"{subdirectory}.csv"
            df.to_csv(os.path.join(sim_dir, csv_filename), index=False)

if __name__ == "__main__":
    main()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


  0%|          | 0/7 [00:00<?, ?it/s]